[Reference](https://python.plainenglish.io/how-to-implement-pagination-using-fastapi-in-python-6d57be902fd5)

In [1]:
# !pip install fastapi
# !pip install uvicorn
# !uvicorn my_pagination:app --reload

# Python Code to connect to MySQL database:

In [3]:
from sqlalchemy import create_engine, text
from sqlalchemy.exc import SQLAlchemyError
from sqlalchemy.orm import sessionmaker
from urllib.parse import quote


def connect_to_db():

    connection_url = "mysql+pymysql://<username>:<password>@<hostname>:3306/<databasename>"

    try:
        engine = create_engine(url=connection_url)

        # create a session
        session = sessionmaker(bind=engine)
        conn = session()

        if session:
            return conn

    except SQLAlchemyError as se:
        print(se)

# Populate fake data to mysql using Faker:

In [4]:
pip install Faker

In [5]:
def insert_fake_data_to_db():
    fake = Faker()
    for i in range(1000):
        name = fake.name()
        id = random.randint(0, 2000)
        dept = random.randint(10, 20)
        dob = fake.date_of_birth()
        loc = fake.country()

        # prepare the insert query
        sql = text("insert into employee(empid, empname, dept, location, dob) values ({},'{}',{},'{}','{}')".format(id, name, dept, loc, dob))

        cnx = connect_to_db()

        cnx.execute(sql)
        cnx.commit()


insert_fake_data_to_db()

# Database Schema using SQLAlchemy:

In [7]:
from sqlalchemy.orm import sessionmaker, declarative_base

Base = declarative_base()

app = FastAPI(title=" A simple pagination learning exercise",
              debug=True)

class Employee(Base):

    __tablename__ = "employee"

    empid = Column(Integer, primary_key=True)
    empname = Column(String)
    dept = Column(Integer)
    location = Column(String)
    dob = Column(Date)

@app.get(path="/api/employees/all", name="Gets all employees")
async def get_all_employees():
    conn = connect_to_db()
    results = conn.query(Employee).all()
    return {"all_employees": results}

In [8]:
from fastapi_pagination import Page, paginate, add_pagination
from pydantic import BaseModel
from fastapi import FastAPI
from sqlalchemy.exc import SQLAlchemyError
from sqlalchemy.orm import sessionmaker, declarative_base
from datetime import date

app = FastAPI(title=" A simple pagination learning exercise",
              debug=True)
add_pagination(app)

class Employee(Base):

    __tablename__ = "employee"

    empid = Column(Integer, primary_key=True)
    empname = Column(String)
    dept = Column(Integer)
    location = Column(String)
    dob = Column(Date)

class EmployeeOut(BaseModel):
    empid: int
    empname: str
    dept: int
    location: str
    dob: date

    class Config:
        orm_mode = True

@app.get(path="/api/employees/all", name="Gets all employees", response_model=Page[EmployeeOut])
async def get_all_employees():
    conn = connect_to_db()
    results = conn.query(Employee).all()
    return paginate(results)

In [2]:
from fastapi import FastAPI

app = FastAPI(title=" A simple pagination learning exercise",
              debug=True)


@app.get(path="/api/hello", name="hello endpoint")
async def hello():
    return {"exercise": "pagination"}